# All Events by District

districtCode=FMA

In [2]:
import requests
import json
import os

apikey = os.environ['FRC_API_KEY']

url = "https://frc-api.firstinspires.org/v3.0/2023/events?districtCode=FMA"

payload={}
headers = {
  'Authorization': 'Basic '+apikey,
  'If-Modified-Since': ''
}

response = requests.request("GET", url, headers=headers, data=payload)
events = response.json()

print(*(events['Events'][i]['code'] for i in range(len(events['Events']))))


MRCMP NJFLA NJROB NJSKI NJTAB NJWAS PABEN PAHAT PAPHI


# Teams by Event

Using eventCode=

In [27]:
url = "https://frc-api.firstinspires.org/v3.0/2023/teams?districtCode=FMA&eventCode=PAPHI"

payload={}
headers = {
  'Authorization': 'Basic '+apikey,
  'If-Modified-Since': ''
}

response = requests.request("GET", url, headers=headers, data=payload)
json = response.json()
print(json["teams"][0]["teamNumber"])

102


## Determine Teams per Event

In [3]:
import requests
import json

eventCodes = []
for i in range(len(events["Events"])):
  eventCodes.append(events["Events"][i]["code"])
  
base_eventCode_url = "https://frc-api.firstinspires.org/v3.0/2023/teams?districtCode=FMA&eventCode="

event_urls = {}
for eventCode in eventCodes:
  event_urls[eventCode] = base_eventCode_url+eventCode

payload={}
headers = {
  'Authorization': 'Basic '+apikey,
  'If-Modified-Since': ''
}

teams_at_events = {}

for eventCode, url in event_urls.items():
  response = requests.request("GET", url, headers=headers, data=payload)
  if response.status_code == 200:
    event_teams = response.json()
    num_teams = len(event_teams["teams"])
    if num_teams > 0:
      for i in range(num_teams):
        if eventCode in teams_at_events:
          teams_at_events[eventCode].append(event_teams["teams"][i]["teamNumber"])
        else:
          teams_at_events[eventCode] = [event_teams["teams"][i]["teamNumber"]]
  else:
    print(f"mlff: get.py: {eventCode} {response.status_code}")


for event, teams in teams_at_events.items():
  print(f"{event}: {teams}")


NJFLA: [11, 41, 56, 75, 102, 193, 219, 222, 223, 484, 555, 752, 1143, 1279, 1672, 1676, 1811, 1923, 1989, 2070, 2458, 2577, 3142, 3515, 4361, 4573, 5438, 5732, 5992, 6860, 6945, 7045, 8117, 8513, 8628, 9116]
NJROB: [87, 293, 303, 316, 714, 1089, 1257, 1403, 1626, 1807, 1812, 2016, 2191, 2495, 2554, 2590, 3314, 4652, 4653, 5624, 5666, 5684, 6897, 7587, 8075, 8130, 8704, 8707, 8714, 9015, 9116]
NJSKI: [25, 41, 75, 204, 293, 752, 1089, 1257, 1279, 1403, 1626, 1672, 1923, 2070, 2495, 2554, 2577, 3340, 3515, 3637, 4361, 4750, 5438, 5992, 6016, 6860, 6897, 7587, 8075, 8513, 8588, 8706, 8714]
NJTAB: [25, 87, 203, 204, 272, 365, 816, 1168, 1647, 1712, 1807, 1812, 2180, 2191, 2559, 2720, 2722, 4342, 4750, 5113, 5457, 5732, 5895, 6226, 6921, 7110, 8130, 8630, 8704, 8721, 9100]
NJWAS: [11, 56, 193, 219, 222, 223, 303, 555, 714, 834, 1143, 1672, 1676, 1811, 1989, 2070, 3142, 3340, 4285, 4361, 4573, 4652, 6943, 6945, 7045, 8117, 8588, 8628, 8706, 8707, 9015]
PABEN: [272, 316, 321, 484, 486, 1391, 1

# Collect Rankings

Rankings for 2019, 2021, 2022

In [88]:
import statistics
url22 = "https://frc-api.firstinspires.org/v3.0/2022/rankings/district?districtCode=FMA"
url19 = "https://frc-api.firstinspires.org/v3.0/2019/rankings/district?districtCode=FMA"
url18 = "https://frc-api.firstinspires.org/v3.0/2018/rankings/district?districtCode=MAR"

rank_year_urls = {2018:url18, 2019:url19, 2022:url22}

payload={}
headers = {
  'Authorization': 'Basic '+apikey,
  'If-Modified-Since': ''
}

teams_and_ranks_by_year = {2018: {}, 2019:{}, 2022:{}}

for year, url in rank_year_urls.items():
  response = requests.request("GET", url, headers=headers, data=payload)
  if response.status_code == 200:
    team_ranks = response.json()
    num_teams = len(team_ranks["districtRanks"])
    if num_teams > 0:
      for i in range(num_teams):
        team_num = team_ranks["districtRanks"][i]["teamNumber"]
        if team_num in teams_and_ranks_by_year[year]:
          teams_and_ranks_by_year[year][team_num].append(team_ranks["districtRanks"][i]["event1Points"])
          teams_and_ranks_by_year[year][team_num].append(team_ranks["districtRanks"][i]["event2Points"])
        else:
          teams_and_ranks_by_year[year][team_num] = [team_ranks["districtRanks"][i]["event1Points"]]
          teams_and_ranks_by_year[year][team_num].append(team_ranks["districtRanks"][i]["event2Points"])
  else:
    print(f"mlff: get.py: {year} {response.status_code}")

sorttup18 = sorted(teams_and_ranks_by_year[2018].items(), reverse=True, key=lambda x:statistics.mean(x[1]))
sort18 = {}
for k in sorttup18:
  if k[0] not in sort18:
    sort18[k[0]] = k[1]
  else:
    sort18[k[0]].append(k[1])

#print(json.dumps(ranks_2019, indent=2))


## Sort Teams per Event by Rank in 2018

In [95]:
for event, teams in teams_at_events.items():  
    print(f"{event}: ")
    for team in sort18:
        if team in teams:
            print(team, sort18[team])
    print()


NJFLA: 
1923 [50.0, 67.0]
56 [69.0, 44.0]
222 [53.0, 53.0]
223 [53.0, 47.0]
1676 [37.0, 46.0]
219 [33.0, 37.0]
1989 [41.0, 28.0]
75 [27.0, 41.0]
6860 [29.0, 30.0]
102 [32.0, 25.0]
41 [24.0, 32.0]
4361 [29.0, 26.0]
5992 [13.0, 42.0]
6945 [11.0, 42.0]
3142 [26.0, 27.0]
1143 [35.0, 17.0]
11 [24.0, 24.0]

NJROB: 
303 [73.0, 70.0]
2590 [63.0, 73.0]
3314 [46.0, 51.0]
316 [54.0, 34.0]
1257 [39.0, 43.0]
2016 [43.0, 26.0]
4653 [24.0, 35.0]
293 [23.0, 34.0]
1403 [23.0, 33.0]
1089 [27.0, 26.0]
1807 [22.0, 31.0]
2495 [7.0, 43.0]

NJSKI: 
25 [72.0, 66.0]
1923 [50.0, 67.0]
1257 [39.0, 43.0]
3637 [38.0, 31.0]
75 [27.0, 41.0]
6860 [29.0, 30.0]
293 [23.0, 34.0]
41 [24.0, 32.0]
1403 [23.0, 33.0]
4361 [29.0, 26.0]
5992 [13.0, 42.0]
1089 [27.0, 26.0]
2495 [7.0, 43.0]

NJTAB: 
25 [72.0, 66.0]
365 [61.0, 50.0]
1712 [57.0, 41.0]
4342 [50.0, 45.0]
2180 [46.0, 40.0]
203 [30.0, 51.0]
7110 [44.0, 29.0]
272 [31.0, 38.0]
5895 [24.0, 43.0]
1168 [9.0, 45.0]
1807 [22.0, 31.0]
6226 [11.0, 39.0]

NJWAS: 
303 [73.0, 70.